In [ ]:
%pip install --upgrade setuptools
%pip install --upgrade gradio
%pip install --upgrade python-dotenv
%pip install --upgrade tabulate

In [ ]:
import json


import sys
from dotenv import dotenv_values

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from examples.coffeeShop import schema as coffeeshop
from typechat import Failure, TypeChatJsonTranslator, TypeChatValidator, create_language_model

In [ ]:
env_vals = dotenv_values()
model = create_language_model(env_vals)
validator = TypeChatValidator(coffeeshop.Cart)
translator = TypeChatJsonTranslator(model, validator, coffeeshop.Cart)

In [ ]:
import pandas
async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        result = result.value
        df = pandas.DataFrame.from_dict(result["items"])
        #return f"Translation Succeeded! ✅\n JSON View \n ``` {json.dumps(result, indent=2)} \n ``` \n"
        return f"Translation Succeeded! ✅\n Coffee Shop Items \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"

def get_examples():
    example_prompts = []
    with open('../examples/coffeeShop/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts


In [ ]:
import setuptools
import gradio as gr

gr.ChatInterface(get_translation, title="☕ Coffee", examples=get_examples()).launch()